In [36]:
import wradlib
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import ogr, osr
import numpy as np

In [37]:
import descartes

In [38]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [39]:
dem = wradlib.io.read_generic_netcdf(r"P:\progress\global_radar_map\GMTED2010_15n030_0125deg.nc")

In [5]:
radars = pd.read_csv(r"P:\progress\global_radar_map\GlobalRadarStations.csv", sep = ";", encoding="cp1252")

In [6]:
lat = dem["variables"]["latitude"]["data"]
lon = dem["variables"]["longitude"]["data"]
elev = dem["variables"]["elevation"]["data"]

In [31]:
lon, lat = np.meshgrid(lon, lat)

In [52]:
# Winkel-Tripel
proj = wradlib.georef.epsg_to_osr(16)

In [53]:
x, y = wradlib.georef.reproject(lon, lat, projection_target=proj)

TypeError: in method 'CoordinateTransformation_TransformPoints', argument 1 of type 'OSRCoordinateTransformationShadow *'

In [7]:
#radpoints = []
#for i in range(len(radars)):
#    point = ogr.Geometry(ogr.wkbPoint)
#    point.AddPoint(radars.x[i], radars.y[i])
#    radpoints.append(point)

In [8]:
def write_geojson(fname, geometry, geom_type):
    """Write GeoJSON file for ogr geometries
    
    Parameters
    ----------
    fname : string
    geometry : ogr Geometry object
    geom_type : ogr Geometry type, 
        e.g. ogr.wkbMultiPolygon, ogr.wkbPolygon, ogr.wkbMultiPoint
    
    """
    outDriver = ogr.GetDriverByName('GeoJSON')

    # Create the output GeoJSON
    outDataSource = outDriver.CreateDataSource(fname)
    outLayer = outDataSource.CreateLayer(fname, geom_type=geom_type)

    # Get the output Layer's Feature Definition
    featureDefn = outLayer.GetLayerDefn()

    # create a new feature
    outFeature = ogr.Feature(featureDefn)

    # Set new geometry
    outFeature.SetGeometry(geometry)

    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)

    # dereference the feature
    outFeature = None

    # Save and close DataSources
    outDataSource = None

In [40]:
radpoints = ogr.Geometry(ogr.wkbMultiPoint)
for i in range(len(radars)):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(radars.x[i], radars.y[i])
    radpoints.AddGeometry(point)
    
source = wradlib.georef.get_default_projection()
transform = osr.CoordinateTransformation(source, winkeltr)

radpoints.Transform(transform)

write_geojson("out/radpoints.geojson", radpoints, ogr.wkbMultiPoint)

RuntimeError: latitude or longitude exceeded limits

In [48]:
lon.max()

179.9375

In [10]:
bufferDistance = 1.5
radbufrs = radpoints.Buffer(bufferDistance)

write_geojson("out/radbufrs.geojson", radbufrs, ogr.wkbMultiPolygon)

In [11]:
#bufferDistance = 1
#radbufrs = []
#for radpoint in radpoints:
#    poly = radpoint.Buffer(bufferDistance)
#    radbufrs.append(poly)    

In [12]:
# Create ring
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(-180, -90)
ring.AddPoint(-180, 90)
ring.AddPoint(180, 90)
ring.AddPoint(180, -90)
ring.AddPoint(-180, -90)

# Create polygon
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

write_geojson("out/clipfrom.geojson", poly, ogr.wkbPolygon)

In [13]:
clipped = poly.Difference(radbufrs)

In [14]:
write_geojson("out/clipped.geojson", clipped, ogr.wkbMultiPolygon)

In [16]:
clipped_dict = eval(clipped.ExportToJson())

In [18]:
clipped_dict["type"]

'MultiPolygon'

In [21]:
from descartes import PolygonPatch
BLUE = '#6699cc'
fig = plt.figure() 
ax = fig.gca() 
ax.add_patch(PolygonPatch(clipped_dict, fc="black", ec="black", alpha=0.5, zorder=2 ))
ax.axis('scaled')

In [24]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, aspect="equal")
plt.pcolormesh(lon, lat, elev, cmap=plt.cm.terrain)
#plt.plot(radars.x, radars.y, "ro")
ax.add_patch(PolygonPatch(clipped_dict, fc="black", ec="black", alpha=0.7, zorder=2 ))
#ax.axis('scaled')